In [2]:
!pip install mysql-connector-python


   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   --------------- ------------------------ 6.3/16.4 MB 36.3 MB/s eta 0:00:01
   --------------------------------- ------ 13.9/16.4 MB 36.4 MB/s eta 0:00:01
   ---------------------------------------- 16.4/16.4 MB 33.2 MB/s eta 0:00:00


In [1]:
import requests
import pandas as pd
import mysql.connector
import json
from datetime import datetime

# ---------------------------
# Load DB config
# ---------------------------
with open(r"C:\Users\Brylle\Desktop\Jupyter Projects\Weather-ETL-with-Python-MySQL\db_config.json") as f:
    config = json.load(f)

# ---------------------------
# Load Weather API config
# ---------------------------
with open(r"C:\Users\Brylle\Desktop\Jupyter Projects\Weather-ETL-with-Python-MySQL\config.json") as f:
    secrets = json.load(f)

API_KEY = secrets["api_key"]

# List of cities to fetch
CITIES = ["Manila", "Tokyo", "New York", "London", "Sydney"]

# ---------------------------
# Extract
# ---------------------------
def extract(cities):
    results = []
    for city in cities:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}"
        response = requests.get(url)
        print(f"Fetching {city}: {response.status_code}")
        if response.status_code == 200:
            results.append(response.json())
        else:
            print(f"❌ Failed for {city}: {response.text}")
    return results

# ---------------------------
# Transform
# ---------------------------
def transform(data_list):
    records = []
    for data in data_list:
        record = {
            "city": data["name"],
            "country": data["sys"]["country"],
            "temperature_c": round(data["main"]["temp"] - 273.15, 2),  # Kelvin → Celsius
            "humidity": data["main"]["humidity"],
            "wind_speed": data["wind"]["speed"],
            "description": data["weather"][0]["description"],
            "timestamp": datetime.now()
        }
        records.append(record)
    return pd.DataFrame(records)

# ---------------------------
# Load
# ---------------------------
def load(df):
    conn = mysql.connector.connect(
        host=config["host"],
        user=config["user"],
        password=config["password"],
        database="weather_db"
    )
    cursor = conn.cursor()

    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO weather_data (city, country, temperature_c, humidity, wind_speed, description, timestamp)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            row["city"],
            row["country"],
            row["temperature_c"],
            row["humidity"],
            row["wind_speed"],
            row["description"],
            row["timestamp"]
        ))

    conn.commit()
    cursor.close()
    conn.close()
    print("✅ Data loaded into MySQL")

# ---------------------------
# Main Script
# ---------------------------
if __name__ == "__main__":
    raw_data_list = extract(CITIES)
    df = transform(raw_data_list)
    print(df)  # preview in Jupyter
    df.to_csv("sample_output.csv", index=False)  # Save to CSV
    load(df)


Fetching Manila: 200
Fetching Tokyo: 200
Fetching New York: 200
Fetching London: 200
Fetching Sydney: 200
       city country  temperature_c  humidity  wind_speed      description  \
0    Manila      PH          29.35        85        0.55    broken clouds   
1     Tokyo      JP          31.70        59        6.84    broken clouds   
2  New York      US          17.32        71        4.93    broken clouds   
3    London      GB          14.05        81        4.20  overcast clouds   
4    Sydney      AU          16.36        93        4.20       light rain   

                   timestamp  
0 2025-09-10 13:23:11.381745  
1 2025-09-10 13:23:11.381752  
2 2025-09-10 13:23:11.381756  
3 2025-09-10 13:23:11.381758  
4 2025-09-10 13:23:11.381760  
✅ Data loaded into MySQL
